# Basic usage example

This example shows how to connect to Granta MI and perform a basic query for impacted substances. It also
demonstrates how to view logging messages returned by the Granta MI server. For more information about the
results of the queries, see the examples in [Impacted Substances](1_Impacted_Substances_Queries/index.rst) and
[Compliance](2_Compliance_Queries/index.rst).

## Connect to Granta MI

First, use the ``ansys.grantami.bomanalytics.Connection`` class to connect to the Granta MI server. The ``Connection``
class uses a fluent interface to build the connection, which is always invoked in the following sequence:

1. Specify your Granta MI Service Layer URL as a parameter to the ``Connection`` class.
2. Specify the authentication method using a ``Connection.with_...()`` method.
3. Use the ``Connection.connect()`` method to finalize the connection.

This returns a connection object, which is called ``cxn`` in these examples.

In [1]:
from ansys.grantami.bomanalytics import Connection

server_url = "http://my_grantami_server/mi_servicelayer"

If you are running your Python script on Windows, you are generally able to use ``.with_autologon()``.

In [2]:
cxn = Connection(server_url).with_autologon().connect()
cxn

<BomServicesClient: url="http://my_grantami_server/mi_servicelayer", maximum_spec_link_depth="unlimited", dbkey="MI_Restricted_Substances">

If the Python script is running on Linux without Kerberos enabled, or you want to use an account other than your
logged-in account, you can specify credentials explicitly.

In [3]:
cxn = Connection(server_url).with_credentials("my_username", "my_password").connect()
cxn

<BomServicesClient: url="http://my_grantami_server/mi_servicelayer", maximum_spec_link_depth="unlimited", dbkey="MI_Restricted_Substances">

OIDC and anonymous authentication methods are also available, but they are beyond the scope of this example.
For more information, see the [ansys-openapi-common](https://github.com/pyansys/openapi-common) package
documentation.

## Construct a query

Queries are also constructed using a fluent interface. However, the ``Query`` constructor takes no arguments. All
query details are specified using ``Query`` methods. To demonstrate this, this example builds a query to
determine all substances present in an ABS material that are impacted by the REACH Candidate List legislation.

First import the ``queries`` module and create a ``MaterialImpactedSubstancesQuery`` object.

In [4]:
from ansys.grantami.bomanalytics import queries

query = queries.MaterialImpactedSubstancesQuery()
query

<MaterialImpactedSubstancesQuery: 0 materials, batch size = 100, 0 legislations>

Now add the material that you want to query by specifying its material ID. (Alternate methods of specifying records
are shown in other examples.)

In [5]:
query = query.with_material_ids(["plastic-abs-high-impact"])
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 0 legislations>

Note that because the ``MaterialImpactedSubstancesQuery`` object has a fluent interface, you receive the same object
back that you started with, but with the material IDs added.

Finally, add the legislation to the query. Legislations are identified by their ``Short title`` attribute.

In [6]:
query = query.with_legislations(["EU REACH - The Candidate List"])
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations>

Fluent interfaces are designed to allow a complex object to be constructed in a single line of code. As such, you can
consolidate the cells above into a single step:

In [7]:
query = queries.MaterialImpactedSubstancesQuery().with_material_ids(["plastic-abs-high-impact"]).with_legislations(["EU REACH - The Candidate List"])  # noqa: E501
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations>

Because the fluent interface can produce very long lines of code, it's necessary to break your query creation code
into multiple lines. The following multi-line format is used throughout the examples. It is functionally equivalent to
the preceding cell:

In [8]:
query = (
    queries.MaterialImpactedSubstancesQuery()
    .with_material_ids(["plastic-abs-high-impact"])
    .with_legislations(["EU REACH - The Candidate List"])
)
query

<MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations>

The multi-line format is the recommended way of creating queries using this API.

## Run a query

Now that you have your ``cxn`` and ``query`` objects, you can use the ``cxn.run()`` method to run the query. This
returns an object that contains the results of the query.

In [9]:
result = cxn.run(query)
result

<MaterialImpactedSubstancesQueryResult: 1 MaterialWithImpactedSubstances results>

## View query results

In the case of ``MaterialImpactedSubstancesQuery``, the results object contains the list of substances present in
the material that are impacted by the specified legislations.

In [10]:
result.impacted_substances

[<ImpactedSubstance: {"cas_number": "15087-24-8", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "25973-55-1", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "36437-37-3", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "36861-47-9", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "3846-71-7", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "3864-99-1", "percent_amount": 2.0}>,
 <ImpactedSubstance: {"cas_number": "872-50-4", "percent_amount": None}>,
 <ImpactedSubstance: {"cas_number": "90481-04-2", "percent_amount": None}>]

## View logged messages

All query results also contain a list of messages returned by the server while running the query. These are
sorted in order of decreasing severity. The same messages are also available in the MI Service Layer log file.

In [11]:
result.messages

[]

Additionally, these messages are available via the standard ``logging`` module using the
``ansys.grantami.bomanalytics`` logger. Alternatively, you can omit the logger name to get the root logger, which
includes messages logged by all packages.

The following code creates a log handler that outputs all 'ansys.grantami.bomanalytics' logger messages with severity
INFO and above to either the terminal or the notebook.

In [12]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ansys.grantami.bomanalytics")

result = cxn.run(query)

INFO:ansys.grantami.bomanalytics:Running query <MaterialImpactedSubstancesQuery: 1 materials, batch size = 100, 1 legislations> with connection <BomServicesClient: url="http://my_grantami_server/mi_servicelayer", maximum_spec_link_depth="unlimited", dbkey="MI_Restricted_Substances">


INFO:ansys.grantami.bomanalytics:No specification-to-specification link depth limit specified. All links will be followed


INFO:ansys.grantami.bomanalytics:Using default table config


INFO:ansys.grantami.bomanalytics:Using default database key (MI_Restricted_Substances)
